Ao fazer uma breve analise no csv, notei que algumas linhas tem algumas divergencias:
1. Comentarios faltando fechar aspas.
2. E as ultimas linhas apresentam colunas divergentes (Extremamente Negativo)

E essas caractaristicas foram tratas no util.cleaning,\
fechando o comentario e trocando o texto por ','\
tornando as colunas corretas por mais que com NaN

In [1]:
from utils.cleaning import fix_data_structure
from pathlib import Path

file_path = Path('data/01_original_data.csv')
fixed_file_path = fix_data_structure(file_path)

📋 Cabeçalho original:
ID,Nome,Idade,Gênero,Localização,Estado Civil,Número de Dependentes,Data de Início do Contrato,Data de Término do Contrato,Valor Mensal do Contrato,Tipo de Serviço,Duração Média das Chamadas,Volume de Dados,Número de Reclamações,Comentários
Colunas esperadas: 15

✅ Linhas corrigidas: 33


In [2]:
import pandas as pd

df = pd.read_csv('data/02_churn_fixed.csv')

df.columns = [
    'id',
    'name',
    'age',
    'gender',
    'location',
    'marital_status',
    'qtd_dependents',
    'start_date',
    'churn_date',
    'price',
    'service',
    'tma',
    'data_volume',
    'qtd_appointments',
    'comments'
]

df.tail(2) # tail pra validar que as ultimas linhas estao corretas

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments
100,1101,Rodrigo Santos,33,Masculino,Belo Horizonte/MG/Minas Gerais,Solteiro,0,2022-09-01,NaN,99.99,Telefonia Fixa,2.2,NaN,NaN,A telefonia fixa dessa empresa é uma vergonha....
101,1102,Carolina Fernandes,29,Feminino,Porto Alegre/RS/Rio Grande do Sul,Solteiro,0,2023-01-15,NaN,89.99,Telefonia Móvel,1.3,NaN,NaN,A telefonia móvel dessa empresa é um completo ...


In [3]:
df['location'].value_counts()

location
São Paulo/SP/São Paulo                 29
Belo Horizonte/MG/Minas Gerais         25
Rio de Janeiro/RJ/Rio de Janeiro       22
Porto Alegre/RS/Rio Grande do Sul      22
Brasília/DF                             3
Rio de Janeiro/RJ/Rio Grande do Sul     1
Name: count, dtype: int64

Aqui vemos que temos sempre a capital de cada estado, então vou simplificar a notação apenas com no UF.

Obs.: Os dados apresentam uma incongruencia em: Rio de Janeiro/RJ/Rio Grande do Sul\
que deveria ser (provavelmente): Rio de Janeiro/RJ/Rio de Janeiro.\
Isso se resolve com a simplificação proposta

In [4]:
df['location'] = [splits[1] for splits in df['location'].str.split('/')]

In [5]:
df.tail(2)

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments
100,1101,Rodrigo Santos,33,Masculino,MG,Solteiro,0,2022-09-01,NaN,99.99,Telefonia Fixa,2.2,NaN,NaN,A telefonia fixa dessa empresa é uma vergonha....
101,1102,Carolina Fernandes,29,Feminino,RS,Solteiro,0,2023-01-15,NaN,89.99,Telefonia Móvel,1.3,NaN,NaN,A telefonia móvel dessa empresa é um completo ...


Visto que temos bastante incongruencia nos dados, vamos aprofundar nas demais colunas

#### id

In [6]:
df['id'].value_counts()

id
1068    2
2       1
3       1
4       1
1       1
       ..
1098    1
1099    1
1100    1
1101    1
1102    1
Name: count, Length: 101, dtype: int64

In [7]:
df[df['id']==1068]

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments
69,1068,Gabriela Almeida,32,Feminino,RJ,Casado,1,2021-03-20,NaN,129.99,Telefonia Fixa,4.2,NaN,NaN,Estou profundamente insatisfeita com o serviço...
70,1068,Julio Santos,28,Masculino,SP,Solteiro,0,2022-09-01,NaN,99.99,Internet,3.9,NaN,NaN,A internet fornecida é simplesmente terrível. ...


Parece que dois clientes estão dividindo o mesmo id, vamos verificar se esses clientes já existem baseado nos nomes

In [8]:
df[df['name']=='Julio Santos']

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments
70,1068,Julio Santos,28,Masculino,SP,Solteiro,0,2022-09-01,NaN,99.99,Internet,3.9,NaN,NaN,A internet fornecida é simplesmente terrível. ...


In [9]:
df[df['name']=='Gabriela Almeida']

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments
69,1068,Gabriela Almeida,32,Feminino,RJ,Casado,1,2021-03-20,NaN,129.99,Telefonia Fixa,4.2,NaN,NaN,Estou profundamente insatisfeita com o serviço...


Parece que não é o caso, vamos esolher um dos dois e adicionar um novo id

In [10]:
df['id'].describe()

count     102.000000
mean      999.392157
std       229.206559
min         1.000000
25%      1021.250000
50%      1047.000000
75%      1074.750000
max      1102.000000
Name: id, dtype: float64

In [11]:
df.loc[70, 'id'] = 1103

Validar se deu certo

In [12]:
df['id'].value_counts()

id
1       1
2       1
3       1
4       1
5       1
       ..
1098    1
1099    1
1100    1
1101    1
1102    1
Name: count, Length: 102, dtype: int64

In [13]:
df['id'].describe()

count     102.000000
mean      999.735294
std       229.336448
min         1.000000
25%      1021.250000
50%      1047.000000
75%      1075.750000
max      1103.000000
Name: id, dtype: float64

#### name

In [14]:
df['name'].value_counts()

name
Carolina Oliveira     3
Lucas Rodrigues       2
Beatriz Oliveira      2
Felipe Santos         2
Mariana Costa         2
                     ..
Patrícia Lima         1
André Silva           1
Amanda Almeida        1
Rodrigo Santos        1
Carolina Fernandes    1
Name: count, Length: 89, dtype: int64

Alguns nome estão repetidos mas isso não é um problema por si só, vamos seguir assim mesmo

#### age

In [15]:
df['age'].value_counts()

age
29    11
31     9
28     8
27     8
39     7
33     7
35     6
37     6
32     5
42     4
34     4
30     4
36     4
41     3
43     3
45     3
40     3
26     2
38     2
55     1
48     1
46     1
Name: count, dtype: int64

Tudo certo

#### gender

In [16]:
df['gender'].value_counts()

gender
Feminino     52
Masculino    50
Name: count, dtype: int64

Certo!

#### marital_status

In [17]:
df['marital_status'].value_counts()

marital_status
Casado        53
Solteiro      45
Divorciado     3
Viúvo          1
Name: count, dtype: int64

Certo!

#### qtd_dependents

In [18]:
df['qtd_dependents'].value_counts() 

qtd_dependents
0    47
1    26
2    25
3     4
Name: count, dtype: int64

#### start_date

In [19]:
years = [date[0] for date in df['start_date'].str.split('-')]
months = [date[1] for date in df['start_date'].str.split('-')]
days = [date[2] for date in df['start_date'].str.split('-')]

print('years:', min(years), max(years))
print('months:', min(months), max(months))
print('days:', min(days), max(days))    

years: 2017 2023
months: 01 12
days: 01 20


Os valores e estruturas estao coerentes com YYYY-MM-DD

#### churn_date

In [20]:
churn_dates = df['churn_date'].dropna()

years = [date[0] for date in churn_dates.str.split('-')]
months = [date[1] for date in churn_dates.str.split('-')]
days = [date[2] for date in churn_dates.str.split('-')]

print('years:', min(years), max(years))
print('months:', min(months), max(months))
print('days:', min(days), max(days))
print('churn_rate:', churn_dates.shape[0]/df.shape[0])

years: 2021 2023
months: 01 09
days: 05 30
churn_rate: 0.0392156862745098


Os valores e estruturas estao coerentes com YYYY-MM-DD

Mas levanta aqui o ponto de que o churn percebido é muito baixo,\
a analise deve focar mais em detratores (baseado nos comentarios) do que no churn em si

#### price

In [21]:
df['price'].value_counts() 

price
99.99     21
109.99    15
89.99     13
149.99    13
129.99    13
119.99     9
139.99     5
179.99     5
159.99     3
79.99      2
189.99     2
69.99      1
Name: count, dtype: int64

Os valores e estruturas estao coerentes

#### service

In [22]:
df['service'].value_counts() 

service
Internet           31
TV a Cabo          25
Telefonia Móvel    23
Telefonia Fixa     23
Name: count, dtype: int64

Os valores e estruturas estao coerentes

#### tma

In [23]:
df['tma'].value_counts() 

tma
1.2    5
4.8    4
4.6    3
1.1    3
1.3    3
2.9    3
3.9    3
3.2    2
6.2    2
4.5    2
5.3    2
6.9    2
1.5    2
2.2    2
2.4    2
5.5    2
4.2    2
5.2    1
5.1    1
6.7    1
4.3    1
5.9    1
7.2    1
5.8    1
6.5    1
7.5    1
1.9    1
3.7    1
2.1    1
3.5    1
2.6    1
2.7    1
1.4    1
2.3    1
2.8    1
Name: count, dtype: int64

Os valores estão coerentes, mas vamos ajustar de minutos para segundos

In [24]:
df['tma'] = df['tma'] * 60
df['tma'].fillna(0, inplace=True)

/tmp/ipykernel_128776/1924232527.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tma'].fillna(0, inplace=True)


In [25]:
df['tma'].describe()

count    102.000000
mean     133.882353
std      139.197082
min        0.000000
25%        0.000000
50%       81.000000
75%      256.500000
max      450.000000
Name: tma, dtype: float64

#### data_volume

In [26]:
df['data_volume'].value_counts() 

data_volume
-         40
2.3 GB     5
2.1 GB     3
1.8 GB     2
3.5 GB     1
2.7 GB     1
3.2 GB     1
2.5 GB     1
Name: count, dtype: int64

Vamos remover o GB para tratar o valor em si, se quisermos

In [27]:
try:
    df['data_volume'] = df['data_volume'].str.replace(' GB','')
    df['data_volume'] = df['data_volume'].str.replace('-','0')
    df['data_volume'] = df['data_volume'].astype(float)
except AttributeError:
    pass

df['data_volume'].fillna(0, inplace=True)
df['data_volume'].describe()

/tmp/ipykernel_128776/771530290.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['data_volume'].fillna(0, inplace=True)


count    102.000000
mean       0.326471
std        0.840312
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.500000
Name: data_volume, dtype: float64

#### qtd_appointments

In [28]:
df['qtd_appointments'].value_counts() 

qtd_appointments
0    37
1    15
-    15
2     2
Name: count, dtype: int64

In [29]:
df['qtd_appointments'] = df['qtd_appointments'].str.replace('-','0')
df['qtd_appointments'] = df['qtd_appointments'].astype(float)
df['qtd_appointments'].value_counts() 

qtd_appointments
0.0    52
1.0    15
2.0     2
Name: count, dtype: int64

Vamos adicionar o atributo explicito de churn no df

In [30]:
df['churn'] = ~ df['churn_date'].isna()

E já vamos bater o olho nesses 4% de Churn

In [32]:
df.to_csv('data/03_churn_cleaned.csv', index=False)